In [1]:
import os
import sys
import datetime
from pathlib import Path
import xml.etree.ElementTree as ET
import logging
import uuid
import time
import shapely
import pandas as pd
import geopandas as gpd
import seaborn
import dask
import numpy as np
import dask.dataframe as dd
from dask.distributed import Client

sys.path.insert(0, '..')
import src.hotspot_utils as util 
import src.process_nearest_hotspots as nearest_process
import src.xml_util as xutil

/env/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
outdir = Path(f"/home/jovyan/s3vt_dask/s3vtdata/workdir_test")

In [3]:
logging.basicConfig(
    format='%(asctime)s [%(levelname)s] %(name)s - %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S',
)
_LOG = logging.getLogger(__name__)

In [4]:
# include n_workers equal or less than the number of core
client = Client(n_workers=8)
client

Client Scheduler: tcp://127.0.0.1:39899 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 16 Memory: 66.57 GB


# Processing Parameter used in Sub-setting Spatial Extent and Temporal Range for Area of Interest
##### The FRP data from nasa, esa, eumetsat and landgate are merged, sub-setted and neareast hotspots csv files are generated based on the parameters in `processing_parameters`  
##### The parameter `chunks` in blocking FRP data to enable multi-processing. If you encounter memory issues then higher the number.
##### The `start_time` and `end_time` can be used to subset for solar_day (3:00-22:00), solar_night (22:00-03:00 with 12 hours offset) and solar_all(0:00-24:00) hours.

In [5]:
processing_parameters = {
    "nasa_frp": "s3://s3vtaustralia/nasa_hotspots_gdf.geojson",
    "esa_frp": "s3://s3vtaustralia/s3vt_hotspots.geojson",
    "eumetsat_frp": "s3://s3vtaustralia/s3vt_eumetsat_hotspots.geojson",
    "landgate_frp": "s3://s3vtaustralia/landgate_hotspots_gdf.geojson",
    "sentinel3_swath_geojson": "s3://s3vtaustralia/sentinel3_swath_gdfs.geojson",
    "dea_frp": None,
    "lon_west": 147.0,
    "lat_south": -38.0,
    "lon_east": 154.0,
    "lat_north": -27.,
    "start_date": "2019-11-01",
    "end_date": "2020-10-08",
    "start_time": "20:00",
    "end_time": "03:00",
    "chunks": 250,
    "compare_field": "solar_night",
    "swath_config_file": Path("/home/jovyan/s3vtconfig.yaml"),
    "outdir": outdir,
    "test": False
}

In [6]:
# This is to generate nearest .csv files. If .csv files already exists then skip this process. Takes around ~5-6 hours in this sandbox environment with 2-core and 16 GB RAM
nearest_hotspots_product_files = nearest_process.process_nearest_points(**processing_parameters)

2021-09-14 01:57:48,015: INFO: Processing FRP Hotspots from GeoJSON files
2021-09-14 01:57:48,026: INFO: Found credentials in environment variables.
2021-09-14 01:57:48,062: INFO: Fetching FRP datasets...
2021-09-14 01:57:48,062: INFO: s3://s3vtaustralia/nasa_hotspots_gdf.geojson exists: skipped download
2021-09-14 01:57:48,063: INFO: s3://s3vtaustralia/s3vt_hotspots.geojson exists: skipped download
2021-09-14 01:57:48,063: INFO: s3://s3vtaustralia/s3vt_eumetsat_hotspots.geojson exists: skipped download
2021-09-14 01:57:48,064: INFO: s3://s3vtaustralia/landgate_hotspots_gdf.geojson exists: skipped download
2021-09-14 01:57:48,064: INFO: dea Hotspots FRP  is None. excluding from analysis.
2021-09-14 01:57:48,064: INFO: Reading...
2021-09-14 01:57:48,065: INFO: reading and subsetting GeoDataFrame for nasa: /home/jovyan/s3vt_dask/s3vtdata/workdir_test/nasa_hotspots_gdf.geojson
2021-09-14 02:00:00,821: INFO: reading and subsetting GeoDataFrame for esa: /home/jovyan/s3vt_dask/s3vtdata/workd

                     latitude  longitude satellite sensor  confidence  power  \
datetime                                                                       
2019-12-04 14:06:00 -30.06377  152.13608   NOAA 20  VIIRS        50.0    7.0   
2019-12-04 14:06:00 -33.95615  150.34776   NOAA 20  VIIRS        50.0    3.5   
2019-12-04 14:06:00 -33.95773  150.35387   NOAA 20  VIIRS        50.0    3.6   
2019-12-04 14:06:00 -33.95880  150.35799   NOAA 20  VIIRS        50.0    3.6   
2019-12-04 14:06:00 -33.96070  150.36525   NOAA 20  VIIRS        50.0    1.6   

                               datetime           solar_day  \
datetime                                                      
2019-12-04 14:06:00 2019-12-04 14:06:00 2019-12-05 00:14:32   
2019-12-04 14:06:00 2019-12-04 14:06:00 2019-12-05 00:07:23   
2019-12-04 14:06:00 2019-12-04 14:06:00 2019-12-05 00:07:24   
2019-12-04 14:06:00 2019-12-04 14:06:00 2019-12-05 00:07:25   
2019-12-04 14:06:00 2019-12-04 14:06:00 2019-12-05 00:07:27  

2021-09-14 03:24:52,081: INFO: Generating satellite swath concatenated GeoDataFrame..
2021-09-14 03:24:52,091: INFO: Found credentials in environment variables.
2021-09-14 03:24:52,399: INFO: sentinel3_swath_gdfs.geojson exists: skipped download


     AcquisitionOfSignalLocal      AcquisitionOfSignalUTC  \
0  2019-11-01T09:32:42.034000  2019-11-01T00:02:42.034000   
1  2019-11-01T09:32:42.034000  2019-11-01T00:02:42.034000   
2  2019-11-01T09:32:42.034000  2019-11-01T00:02:42.034000   
3  2019-11-01T09:35:42.034000  2019-11-01T00:05:42.034000   
4  2019-11-01T09:35:42.034000  2019-11-01T00:05:42.034000   

              LossOfSignalUTC                 TransitTime    Satellite Sensor  \
0  2019-11-01T00:05:42.034000  2019-11-01T00:04:12.034000  Sentinel_3A  SLSTR   
1  2019-11-01T00:05:42.034000  2019-11-01T00:04:12.034000  Sentinel_3A  SLSTR   
2  2019-11-01T00:05:42.034000  2019-11-01T00:04:12.034000  Sentinel_3A  SLSTR   
3  2019-11-01T00:08:42.034000  2019-11-01T00:07:12.034000  Sentinel_3A  SLSTR   
4  2019-11-01T00:08:42.034000  2019-11-01T00:07:12.034000  Sentinel_3A  SLSTR   

  OrbitNumber  OrbitHeight        Node  \
0       19296        814.5  descending   
1       19296        814.5  descending   
2       19296       

2021-09-14 03:29:39,295: INFO: /home/jovyan/s3vt_dask/s3vtdata/workdir_test/swaths_154_147_20191101_20201008 has been archived at /home/jovyan/s3vt_dask/s3vtdata/workdir_test/swaths_154_147_20191101_20201008.
2021-09-14 03:29:40,038: INFO: Generating neareast hotspots...


## Nearest Hotspots DataFrame merged from neareast hotspots csv files

In [7]:
# csv directory is where nearest hotspots csv files are stored. 
csv_directory = outdir
print(csv_directory)
# This is read all the .csv files if name starts with `nearest_points` and ends with `compare_field` value from processing parameters.
nearest_hotspots_csv_files = [
    fp for fp in csv_directory.iterdir()
    if (fp.name.startswith("nearest_points"))
    and (fp.name.endswith("csv"))
]

/home/jovyan/s3vt_dask/s3vtdata/workdir_test


In [8]:
# nearest points csv files that will be used analysis from here on.
nearest_hotspots_csv_files

[PosixPath('/home/jovyan/s3vt_dask/s3vtdata/workdir_test/nearest_points_TERRA_MODIS_LANDGATE_2000_0300.csv'),
 PosixPath('/home/jovyan/s3vt_dask/s3vtdata/workdir_test/nearest_points_SUOMI NPP_VIIRS_NASA1_2000_0300.csv'),
 PosixPath('/home/jovyan/s3vt_dask/s3vtdata/workdir_test/nearest_points_SENTINEL_3B_SLSTR_ESA_2000_0300.csv'),
 PosixPath('/home/jovyan/s3vt_dask/s3vtdata/workdir_test/nearest_points_TERRA_MODIS_NASA6.03_2000_0300.csv'),
 PosixPath('/home/jovyan/s3vt_dask/s3vtdata/workdir_test/nearest_points_NOAA 20_VIIRS_NASA2.0NRT_2000_0300.csv'),
 PosixPath('/home/jovyan/s3vt_dask/s3vtdata/workdir_test/nearest_points_SENTINEL_3B_SLSTR_EUMETSAT_2000_0300.csv'),
 PosixPath('/home/jovyan/s3vt_dask/s3vtdata/workdir_test/nearest_points_NOAA 20_VIIRS_LANDGATE_2000_0300.csv'),
 PosixPath('/home/jovyan/s3vt_dask/s3vtdata/workdir_test/nearest_points_SUOMI NPP_VIIRS_LANDGATE_2000_0300.csv'),
 PosixPath('/home/jovyan/s3vt_dask/s3vtdata/workdir_test/nearest_points_AQUA_MODIS_LANDGATE_2000_0300.

In [9]:
# returns a dask DataFrame with index set at column `compare_field` from processing_parameters.
nearest_points_ddf = util.csv_to_dataframe(nearest_hotspots_csv_files, processing_parameters["compare_field"])

In [10]:
nearest_points_ddf.head()

,Unnamed: 0,latitude,longitude,satellite,sensor,confidence,power,datetime,solar_day,satellite_sensor_product,...,2_power,2_datetime,2_solar_day,2_satellite_sensor_product,2_geometry,2_solar_night,dist,dist_m,timedelta,count
solar_night,,,,,,,,,,,,,,,,,,,,,
2019-11-01 10:01:39,219939,-29.863303,153.123948,SENTINEL_3B,SLSTR,-1.0,0.46,2019-11-01 11:49:10,2019-11-01 22:01:39,SENTINEL_3B_SLSTR_ESA,...,-1.0,2019-11-01 15:02:00,2019-11-02 01:14:34,AQUA_MODIS_LANDGATE,POINT (153.145 -29.865),2019-11-01 13:14:34,0.021121,2042.708227,0 days 03:12:50,1
2019-11-01 10:01:39,140153,-29.845804,153.119394,SENTINEL_3B,SLSTR,-1.0,0.32,2019-11-01 11:49:11,2019-11-01 22:01:39,SENTINEL_3B_SLSTR_ESA,...,-1.0,2019-11-01 15:13:00,2019-11-02 01:25:33,SUOMI NPP_VIIRS_LANDGATE,POINT (153.13808 -29.86438),2019-11-01 13:25:33,0.026349,2701.991024,0 days 03:23:49,1
2019-11-01 10:01:39,140152,-29.863303,153.123948,SENTINEL_3B,SLSTR,-1.0,0.46,2019-11-01 11:49:10,2019-11-01 22:01:39,SENTINEL_3B_SLSTR_ESA,...,-1.0,2019-11-01 15:13:00,2019-11-02 01:25:31,SUOMI NPP_VIIRS_LANDGATE,POINT (153.13068 -29.87558),2019-11-01 13:25:31,0.014002,1563.718457,0 days 03:23:50,1
2019-11-01 10:01:39,140151,-29.873943,153.123579,SENTINEL_3B,SLSTR,-1.0,0.55,2019-11-01 11:49:10,2019-11-01 22:01:39,SENTINEL_3B_SLSTR_ESA,...,-1.0,2019-11-01 15:13:00,2019-11-02 01:25:31,SUOMI NPP_VIIRS_LANDGATE,POINT (153.13068 -29.87558),2019-11-01 13:25:31,0.007288,752.346436,0 days 03:23:50,1
2019-11-01 10:01:39,140150,-29.845804,153.119394,SENTINEL_3B,SLSTR,-1.0,0.32,2019-11-01 11:49:11,2019-11-01 22:01:39,SENTINEL_3B_SLSTR_ESA,...,-1.0,2019-11-01 15:13:00,2019-11-02 01:25:33,SUOMI NPP_VIIRS_LANDGATE,POINT (153.13808 -29.86438),2019-11-01 13:25:33,0.026349,2701.991024,0 days 03:23:49,1


# Results
## Co-occurrence metrics

In [11]:
region_alias = "nsw"
output_directory = processing_parameters["outdir"]
comparison_prefix = (
    f"{processing_parameters['start_date'].replace('-', '')}"
    f"_{processing_parameters['end_date'].replace('-', '')}"
    f"_{processing_parameters['start_time'].replace(':','')}"
    f"_{processing_parameters['end_time'].replace(':','')}"
    f"_{region_alias}"
)

In [12]:
# set the nearest distance threshold between two hotspots to confine the analysis within the distance threshold.  
dist_threshold = 5000  # units in meters

In [13]:
nearest_ddf_dist_subset = client.persist(nearest_points_ddf[nearest_points_ddf["dist_m"] < dist_threshold])

In [14]:
# Count of hotspot matches < dist_threshold
numerator = util.dask_pivot_table(
    nearest_ddf_dist_subset,
    index="2_satellite_sensor_product",
    column="satellite_sensor_product",
    values="count",
    aggfunc="count"
).compute()

In [15]:
numerator.astype(int).to_csv(output_directory.joinpath(f"{comparison_prefix}_matches_{dist_threshold}.csv"))
numerator.astype(int)

satellite_sensor_product,SENTINEL_3B_SLSTR_ESA,SENTINEL_3A_SLSTR_ESA,TERRA_MODIS_NASA6.03,TERRA_MODIS_LANDGATE,AQUA_MODIS_LANDGATE,AQUA_MODIS_NASA6.03,SUOMI NPP_VIIRS_LANDGATE,SUOMI NPP_VIIRS_NASA1,NOAA 20_VIIRS_LANDGATE,SENTINEL_3B_SLSTR_EUMETSAT,SENTINEL_3A_SLSTR_EUMETSAT,NOAA 20_VIIRS_NASA2.0NRT
2_satellite_sensor_product,,,,,,,,,,,,
AQUA_MODIS_LANDGATE,14561,11333,18265,66963,104576,18483,356982,250393,300398,5235,5032,165927
SUOMI NPP_VIIRS_LANDGATE,21869,13367,22239,80448,90862,20105,466759,313044,339646,12010,8176,213186
SENTINEL_3A_SLSTR_ESA,66,19121,4313,17185,14320,2430,50880,49846,58000,0,9267,23453
SENTINEL_3B_SLSTR_ESA,27853,34,4625,16613,16398,4721,94166,68875,69551,15532,0,30998
AQUA_MODIS_NASA6.03,16081,9507,16342,58536,94563,21512,343408,238800,276722,7600,4592,163821
SUOMI NPP_VIIRS_NASA1,26212,18320,26511,97206,103344,21360,446228,350945,388019,14237,8830,230393
TERRA_MODIS_NASA6.03,20394,13267,27278,95346,80168,16149,341708,257930,310715,11914,6852,177348
TERRA_MODIS_LANDGATE,22719,15328,27155,100950,88936,17897,381335,288733,345165,13296,7872,199289
NOAA 20_VIIRS_LANDGATE,20948,14530,22077,81333,80809,14468,353341,271257,416082,10497,7035,199234


In [16]:
# Count of hotspot matches - total  
denominator = util.dask_pivot_table(
        nearest_points_ddf,
        index="2_satellite_sensor_product",
        column="satellite_sensor_product",
        values="count",
        aggfunc="count",
    ).compute()

In [17]:
denominator.astype(int).to_csv(output_directory.joinpath(f"{comparison_prefix}_matches_count.csv"))
denominator.astype(int)

satellite_sensor_product,SENTINEL_3B_SLSTR_ESA,SENTINEL_3A_SLSTR_ESA,TERRA_MODIS_NASA6.03,TERRA_MODIS_LANDGATE,AQUA_MODIS_LANDGATE,AQUA_MODIS_NASA6.03,SUOMI NPP_VIIRS_LANDGATE,SUOMI NPP_VIIRS_NASA1,NOAA 20_VIIRS_LANDGATE,SENTINEL_3B_SLSTR_EUMETSAT,SENTINEL_3A_SLSTR_EUMETSAT,NOAA 20_VIIRS_NASA2.0NRT
2_satellite_sensor_product,,,,,,,,,,,,
AQUA_MODIS_LANDGATE,23301,15697,23804,88004,104576,19932,425530,315970,374549,12413,7264,223206
SUOMI NPP_VIIRS_LANDGATE,27853,14919,25581,93863,101413,21321,466759,338697,397998,15859,9028,247119
SENTINEL_3A_SLSTR_ESA,68,19121,4517,18074,16034,2786,58290,55922,62654,0,9267,26635
SENTINEL_3B_SLSTR_ESA,27853,88,4819,17452,17430,5155,102192,75775,78336,15532,0,37880
AQUA_MODIS_NASA6.03,26502,16323,24311,89697,103922,21512,427020,327532,365744,15324,8016,226415
SUOMI NPP_VIIRS_NASA1,27853,19121,27210,100004,104080,21498,446928,350945,398067,15532,9267,239827
TERRA_MODIS_NASA6.03,27759,18108,27278,100093,101758,20755,441797,344537,397569,15506,8897,237708
TERRA_MODIS_LANDGATE,27772,18108,27275,100950,102225,20764,456338,345405,411240,15807,8986,243981
NOAA 20_VIIRS_LANDGATE,24165,17744,26815,99240,104100,20617,454945,341159,416082,12402,8591,238970


In [18]:
# Difference of matched points closer than 5000m
difference = denominator - numerator

In [19]:
difference.astype(int).to_csv(output_directory.joinpath(f"{comparison_prefix}_count_difference.csv"))
difference.astype(int)

satellite_sensor_product,SENTINEL_3B_SLSTR_ESA,SENTINEL_3A_SLSTR_ESA,TERRA_MODIS_NASA6.03,TERRA_MODIS_LANDGATE,AQUA_MODIS_LANDGATE,AQUA_MODIS_NASA6.03,SUOMI NPP_VIIRS_LANDGATE,SUOMI NPP_VIIRS_NASA1,NOAA 20_VIIRS_LANDGATE,SENTINEL_3B_SLSTR_EUMETSAT,SENTINEL_3A_SLSTR_EUMETSAT,NOAA 20_VIIRS_NASA2.0NRT
2_satellite_sensor_product,,,,,,,,,,,,
AQUA_MODIS_LANDGATE,8740,4364,5539,21041,0,1449,68548,65577,74151,7178,2232,57279
SUOMI NPP_VIIRS_LANDGATE,5984,1552,3342,13415,10551,1216,0,25653,58352,3849,852,33933
SENTINEL_3A_SLSTR_ESA,2,0,204,889,1714,356,7410,6076,4654,0,0,3182
SENTINEL_3B_SLSTR_ESA,0,54,194,839,1032,434,8026,6900,8785,0,0,6882
AQUA_MODIS_NASA6.03,10421,6816,7969,31161,9359,0,83612,88732,89022,7724,3424,62594
SUOMI NPP_VIIRS_NASA1,1641,801,699,2798,736,138,700,0,10048,1295,437,9434
TERRA_MODIS_NASA6.03,7365,4841,0,4747,21590,4606,100089,86607,86854,3592,2045,60360
TERRA_MODIS_LANDGATE,5053,2780,120,0,13289,2867,75003,56672,66075,2511,1114,44692
NOAA 20_VIIRS_LANDGATE,3217,3214,4738,17907,23291,6149,101604,69902,0,1905,1556,39736


In [20]:
# Percentage of matched points closer than dist_threshold
percentage = (numerator / denominator) * 100
percentage = np.round(percentage, 2)

In [21]:
percentage.to_csv(output_directory.joinpath(f"{comparison_prefix}_percentage.csv"))
percentage

satellite_sensor_product,SENTINEL_3B_SLSTR_ESA,SENTINEL_3A_SLSTR_ESA,TERRA_MODIS_NASA6.03,TERRA_MODIS_LANDGATE,AQUA_MODIS_LANDGATE,AQUA_MODIS_NASA6.03,SUOMI NPP_VIIRS_LANDGATE,SUOMI NPP_VIIRS_NASA1,NOAA 20_VIIRS_LANDGATE,SENTINEL_3B_SLSTR_EUMETSAT,SENTINEL_3A_SLSTR_EUMETSAT,NOAA 20_VIIRS_NASA2.0NRT
2_satellite_sensor_product,,,,,,,,,,,,
AQUA_MODIS_LANDGATE,62.49,72.20,76.73,76.09,100.00,92.73,83.89,79.25,80.20,42.17,69.27,74.34
SUOMI NPP_VIIRS_LANDGATE,78.52,89.60,86.94,85.71,89.60,94.30,100.00,92.43,85.34,75.73,90.56,86.27
SENTINEL_3A_SLSTR_ESA,97.06,100.00,95.48,95.08,89.31,87.22,87.29,89.13,92.57,NaN,100.00,88.05
SENTINEL_3B_SLSTR_ESA,100.00,38.64,95.97,95.19,94.08,91.58,92.15,90.89,88.79,100.00,NaN,81.83
AQUA_MODIS_NASA6.03,60.68,58.24,67.22,65.26,90.99,100.00,80.42,72.91,75.66,49.60,57.29,72.35
SUOMI NPP_VIIRS_NASA1,94.11,95.81,97.43,97.20,99.29,99.36,99.84,100.00,97.48,91.66,95.28,96.07
TERRA_MODIS_NASA6.03,73.47,73.27,100.00,95.26,78.78,77.81,77.35,74.86,78.15,76.83,77.01,74.61
TERRA_MODIS_LANDGATE,81.81,84.65,99.56,100.00,87.00,86.19,83.56,83.59,83.93,84.11,87.60,81.68
NOAA 20_VIIRS_LANDGATE,86.69,81.89,82.33,81.96,77.63,70.18,77.67,79.51,100.00,84.64,81.89,83.37


In [22]:
# Maximum time between match points < dist_threshold
timemax = util.pandas_pivot_table(
    nearest_ddf_dist_subset.compute(),
    index=["satellite_sensor_product"],
    columns=["2_satellite_sensor_product"],
    values=["timedelta"],
    aggfunc={"timedelta": np.max}
    
)

In [23]:
timemax.to_csv(output_directory.joinpath(f"{comparison_prefix}_max_time_matched_points.csv"))
timemax

timedelta                      \
2_satellite_sensor_product AQUA_MODIS_LANDGATE AQUA_MODIS_NASA6.03   
satellite_sensor_product                                             
AQUA_MODIS_LANDGATE            0 days 01:47:00     0 days 01:44:00   
AQUA_MODIS_NASA6.03            0 days 01:44:00     0 days 00:00:00   
NOAA 20_VIIRS_LANDGATE         0 days 01:30:00     0 days 01:28:00   
NOAA 20_VIIRS_NASA2.0NRT       0 days 01:55:00     0 days 01:51:00   
SENTINEL_3A_SLSTR_ESA          0 days 03:58:44     0 days 04:00:12   
SENTINEL_3A_SLSTR_EUMETSAT     0 days 03:58:44     0 days 03:53:42   
SENTINEL_3B_SLSTR_ESA          0 days 03:55:46     0 days 03:45:39   
SENTINEL_3B_SLSTR_EUMETSAT     0 days 03:51:40     0 days 03:45:39   
SUOMI NPP_VIIRS_LANDGATE       0 days 01:52:00     0 days 01:48:00   
SUOMI NPP_VIIRS_NASA1          0 days 02:09:00     0 days 02:04:00   
TERRA_MODIS_LANDGATE           0 days 03:28:00     0 days 03:24:00   
TERRA_MODIS_NASA6.03           0 days 03:22:00     0 days 03:17:00   

                                                                            \
2_satellite_sensor_product NOAA 20_VIIRS_LANDGATE NOAA 20_VIIRS_NASA2.0NRT   
satellite_sensor_product                                                     
AQUA_MODIS_LANDGATE               0 days 01:30:00          0 days 01:55:00   
AQUA_MODIS_NASA6.03               0 days 01:28:00          0 days 01:51:00   
NOAA 20_VIIRS_LANDGATE            0 days 01:44:00          0 days 01:45:00   
NOAA 20_VIIRS_NASA2.0NRT          0 days 01:45:00          0 days 00:00:00   
SENTINEL_3A_SLSTR_ESA             0 days 04:03:23          0 days 04:01:25   
SENTINEL_3A_SLSTR_EUMETSAT        0 days 04:03:23          0 days 04:01:25   
SENTINEL_3B_SLSTR_ESA             0 days 03:53:03          0 days 03:51:03   
SENTINEL_3B_SLSTR_EUMETSAT        0 days 03:58:08          0 days 03:57:44   
SUOMI NPP_VIIRS_LANDGATE          0 days 00:58:00          0 days 01:00:00   
SUOMI NPP_VIIRS_NASA1             0 days 01:00:00          0 days 00:56:00   
TERRA_MODIS_LANDGATE              0 days 04:06:00          0 days 03:55:00   
TERRA_MODIS_NASA6.03              0 days 04:02:00          0 days 03:38:00   

                                                                             \
2_satellite_sensor_product SENTINEL_3A_SLSTR_ESA SENTINEL_3A_SLSTR_EUMETSAT   
satellite_sensor_product                                                      
AQUA_MODIS_LANDGATE              0 days 03:58:43            0 days 03:58:43   
AQUA_MODIS_NASA6.03              0 days 04:00:12            0 days 03:53:42   
NOAA 20_VIIRS_LANDGATE           0 days 04:03:30            0 days 04:03:30   
NOAA 20_VIIRS_NASA2.0NRT         0 days 04:01:30            0 days 04:01:29   
SENTINEL_3A_SLSTR_ESA            0 days 00:00:01            0 days 00:00:01   
SENTINEL_3A_SLSTR_EUMETSAT       0 days 00:00:01            0 days 00:00:01   
SENTINEL_3B_SLSTR_ESA            0 days 00:39:07                        NaT   
SENTINEL_3B_SLSTR_EUMETSAT                   NaT                        NaT   
SUOMI NPP_VIIRS_LANDGATE         0 days 03:56:33            0 days 04:03:31   
SUOMI NPP_VIIRS_NASA1            0 days 04:04:17            0 days 03:50:42   
TERRA_MODIS_LANDGATE             0 days 01:44:42            0 days 01:44:42   
TERRA_MODIS_NASA6.03             0 days 01:34:39            0 days 01:34:39   

                                                                             \
2_satellite_sensor_product SENTINEL_3B_SLSTR_ESA SENTINEL_3B_SLSTR_EUMETSAT   
satellite_sensor_product                                                      
AQUA_MODIS_LANDGATE              0 days 03:55:46            0 days 03:51:39   
AQUA_MODIS_NASA6.03              0 days 03:45:39            0 days 03:45:39   
NOAA 20_VIIRS_LANDGATE           0 days 03:53:03            0 days 03:58:44   
NOAA 20_VIIRS_NASA2.0NRT         0 days 03:51:03            0 days 03:57:44   
SENTINEL_3A_SLSTR_ESA            0 days 00:39:07                        NaT 

In [24]:
# Minimum time between match points < dist_threshold
timemin = util.pandas_pivot_table(
    nearest_ddf_dist_subset.compute(),
    index=["satellite_sensor_product"],
    columns=["2_satellite_sensor_product"],
    values=["timedelta"],
    aggfunc={"timedelta": np.min}
    
)

In [25]:
timemin.to_csv(output_directory.joinpath(f"{comparison_prefix}_min_time_matched_points.csv"))
timemin

timedelta                      \
2_satellite_sensor_product AQUA_MODIS_LANDGATE AQUA_MODIS_NASA6.03   
satellite_sensor_product                                             
AQUA_MODIS_LANDGATE            0 days 00:00:00     0 days 00:00:00   
AQUA_MODIS_NASA6.03            0 days 00:00:00     0 days 00:00:00   
NOAA 20_VIIRS_LANDGATE         0 days 00:00:00     0 days 00:00:00   
NOAA 20_VIIRS_NASA2.0NRT       0 days 00:04:00     0 days 00:00:00   
SENTINEL_3A_SLSTR_ESA          0 days 01:37:24     0 days 01:56:32   
SENTINEL_3A_SLSTR_EUMETSAT     0 days 01:37:24     0 days 02:01:42   
SENTINEL_3B_SLSTR_ESA          0 days 01:54:38     0 days 01:57:38   
SENTINEL_3B_SLSTR_EUMETSAT     0 days 01:54:38     0 days 01:57:38   
SUOMI NPP_VIIRS_LANDGATE       0 days 00:00:00     0 days 00:02:00   
SUOMI NPP_VIIRS_NASA1          0 days 00:00:00     0 days 00:00:00   
TERRA_MODIS_LANDGATE           0 days 01:15:00     0 days 01:24:00   
TERRA_MODIS_NASA6.03           0 days 01:24:00     0 days 01:33:00   

                                                                            \
2_satellite_sensor_product NOAA 20_VIIRS_LANDGATE NOAA 20_VIIRS_NASA2.0NRT   
satellite_sensor_product                                                     
AQUA_MODIS_LANDGATE               0 days 00:00:00          0 days 00:04:00   
AQUA_MODIS_NASA6.03               0 days 00:00:00          0 days 00:00:00   
NOAA 20_VIIRS_LANDGATE            0 days 00:00:00          0 days 00:00:00   
NOAA 20_VIIRS_NASA2.0NRT          0 days 00:00:00          0 days 00:00:00   
SENTINEL_3A_SLSTR_ESA             0 days 01:08:08          0 days 01:43:59   
SENTINEL_3A_SLSTR_EUMETSAT        0 days 01:57:54          0 days 01:43:59   
SENTINEL_3B_SLSTR_ESA             0 days 01:39:25          0 days 01:31:41   
SENTINEL_3B_SLSTR_EUMETSAT        0 days 01:39:25          0 days 01:31:41   
SUOMI NPP_VIIRS_LANDGATE          0 days 00:44:00          0 days 00:40:00   
SUOMI NPP_VIIRS_NASA1             0 days 00:39:00          0 days 00:44:00   
TERRA_MODIS_LANDGATE              0 days 00:32:00          0 days 00:01:00   
TERRA_MODIS_NASA6.03              0 days 01:07:00          0 days 00:06:00   

                                                                             \
2_satellite_sensor_product SENTINEL_3A_SLSTR_ESA SENTINEL_3A_SLSTR_EUMETSAT   
satellite_sensor_product                                                      
AQUA_MODIS_LANDGATE              0 days 01:37:24            0 days 01:37:24   
AQUA_MODIS_NASA6.03              0 days 01:56:33            0 days 02:01:42   
NOAA 20_VIIRS_LANDGATE           0 days 01:08:08            0 days 01:57:54   
NOAA 20_VIIRS_NASA2.0NRT         0 days 01:43:59            0 days 01:43:59   
SENTINEL_3A_SLSTR_ESA            0 days 00:00:00            0 days 00:00:00   
SENTINEL_3A_SLSTR_EUMETSAT       0 days 00:00:00            0 days 00:00:00   
SENTINEL_3B_SLSTR_ESA            0 days 00:39:05                        NaT   
SENTINEL_3B_SLSTR_EUMETSAT                   NaT                        NaT   
SUOMI NPP_VIIRS_LANDGATE         0 days 01:56:08            0 days 01:52:11   
SUOMI NPP_VIIRS_NASA1            0 days 01:15:54            0 days 01:15:54   
TERRA_MODIS_LANDGATE             0 days 00:00:10            0 days 00:00:10   
TERRA_MODIS_NASA6.03             0 days 00:00:20            0 days 00:02:32   

                                                                             \
2_satellite_sensor_product SENTINEL_3B_SLSTR_ESA SENTINEL_3B_SLSTR_EUMETSAT   
satellite_sensor_product                                                      
AQUA_MODIS_LANDGATE              0 days 01:54:38            0 days 01:54:38   
AQUA_MODIS_NASA6.03              0 days 01:57:38            0 days 01:57:38   
NOAA 20_VIIRS_LANDGATE           0 days 01:39:25            0 days 01:39:25   
NOAA 20_VIIRS_NASA2.0NRT         0 days 01:31:41            0 days 01:31:41   
SENTINEL_3A_SLSTR_ESA            0 days 00:39:06                        NaT 

In [26]:
# Average distance (m) between matched points < dist_threshold
averagedist = util.dask_pivot_table(
    nearest_ddf_dist_subset,
    index="2_satellite_sensor_product",
    column="satellite_sensor_product",
    values="dist_m",
    aggfunc="mean",
).compute()

In [27]:
averagedist = np.round(averagedist, 2)
averagedist.to_csv(output_directory.joinpath(f"{comparison_prefix}_average_distance_{dist_threshold}m.csv"))
averagedist

satellite_sensor_product,SENTINEL_3B_SLSTR_ESA,SENTINEL_3A_SLSTR_ESA,TERRA_MODIS_NASA6.03,TERRA_MODIS_LANDGATE,AQUA_MODIS_LANDGATE,AQUA_MODIS_NASA6.03,SUOMI NPP_VIIRS_LANDGATE,SUOMI NPP_VIIRS_NASA1,NOAA 20_VIIRS_LANDGATE,SENTINEL_3B_SLSTR_EUMETSAT,SENTINEL_3A_SLSTR_EUMETSAT,NOAA 20_VIIRS_NASA2.0NRT
2_satellite_sensor_product,,,,,,,,,,,,
AQUA_MODIS_LANDGATE,1419.63,1258.49,955.80,1001.53,0.00,118.97,810.65,925.11,843.19,1976.67,1308.89,956.40
SUOMI NPP_VIIRS_LANDGATE,1051.73,850.16,477.09,610.54,700.95,506.36,0.00,432.26,464.33,1101.82,730.41,614.64
SENTINEL_3A_SLSTR_ESA,1086.27,0.00,508.89,596.35,797.42,627.02,617.93,600.95,582.21,NaN,0.44,636.41
SENTINEL_3B_SLSTR_ESA,0.00,659.64,536.18,670.89,742.71,607.58,609.46,617.94,630.36,1.25,NaN,636.05
AQUA_MODIS_NASA6.03,1682.43,1682.53,1299.64,1484.71,900.39,0.00,1177.67,1303.12,1238.23,1885.11,1679.14,1293.85
SUOMI NPP_VIIRS_NASA1,815.67,573.29,332.45,458.55,533.41,342.64,204.19,0.00,339.66,861.97,526.26,368.28
TERRA_MODIS_NASA6.03,1660.44,1426.92,0.00,568.03,1376.21,1230.38,1142.27,1180.87,1076.44,1647.50,1337.96,1176.36
TERRA_MODIS_LANDGATE,1340.03,1065.55,123.47,0.00,981.41,1012.14,905.70,921.33,836.14,1263.02,971.23,908.61
NOAA 20_VIIRS_LANDGATE,1076.07,743.90,477.04,599.29,658.43,435.48,413.14,545.03,0.00,1143.51,717.01,443.42


In [ ]:
client.close() # close dask.distributed client